**TO DO:** Make a copy of this notebook in your own Google drive and edit the copy.

**TO DO:** Download the data at the following link https://stanfordmlgroup.github.io/competitions/mrnet/ and upload it to your Google Drive.

This will download a folder named 'data'.

*   The dataset consists of 1,250 knee MRIs with image level labels.
*   The training data consists of 1,130 MRIs and the validation data consists of 120 MRIs.
*   They are labelled as abnormal, having an acl tear and/or meniscus tear.
*   Each MRI exam includes data from the axial, coronal and sagittal plane.
*   Axial is a Proton-Density series, coronal is a T1-weighted series and sagittal is T2-weighted series.






Go to "Edit" on the toolbar, then "Notebook Settings" and change the hardware accelerator to GPU.


#Mount Google Drive to access your data



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


The following code uses a python library named 'torchsample'. This is not installed in Google Colab. We can import it by running the commands in the following cell. The exclamation mark communicates to Google Colab to run the commands in the terminal rather than in Python in the current notebook.


In [ ]:
!pip install -e git+https://github.com/ncullen93/torchsample.git#egg=torchsample
!pip install visdom
!pip install nibabel
!pip install h5py
!pip install torchsample
!pip install tensorboardX

Obtaining torchsample from git+https://github.com/ncullen93/torchsample.git#egg=torchsample
  Cloning https://github.com/ncullen93/torchsample.git to ./src/torchsample
  Running command git clone -q https://github.com/ncullen93/torchsample.git /content/src/torchsample
  Running setup.py develop for torchsample
     |████████████████████████████████| 686kB 36.0MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=77a6c3042253eaa24ae4b444ee507b9276e1f045b60e021292a0704d9464d797
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5712 sha256=629b5e7acd4497ab2f381e1335fb6c43e1c2db376d1c859992479eb066a70103
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
     |████████████

In [ ]:
#import all libraries
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
from src.torchsample.torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics

#Define your model
The model is defined in the class 'Net'. The 'init' function initialises the architecture of the model.

The line of code; ```self.pretrained_model = models.resnet18(pretrained=True)``` initialises a pre-trained ResNet18, pre-trained on the ImageNet Dataset. This initialises the weights of the model with the weights for a ResNet18 model that was trained on the ImageNet dataset. This speeds up training.

The line of code ```self.classifer = nn.Linear(1000, 1)``` is a fully connected layer that makes the final prediction.

After the model is initialised, the forward function is called iteratively throughout the training process. The output size of each line is shown in the code.

More information con defining models can be found at https://pytorch.org/vision/stable/models.html

In [ ]:
models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
list( models.resnet18(pretrained=True).children())[:-1]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [ ]:
#modify the last fully connected layer to output (1) instead of (1000)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model =  nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-1]  )    # delete the last fc layer.
        self.classifer = nn.Linear(512, 1)


    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x) #output size (s, 512)
        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 512)
        output = self.classifer(output.squeeze(2).squeeze(2)) #output size (1)

        return output

In [ ]:
#add another fully connected layer to convert output (1,1000) to (1)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model =  models.resnet18(pretrained=True)   # delete the last fc layer.
        self.classifer = nn.Linear(1000, 1)


    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x) #output size (s, 1000)
        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 1000)
        output =nn.ReLU()(output)
        output = self.classifer(output) #output size (1)

        return output

In [ ]:
mod=Net()
mod

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Net(
  (pretrained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

TO NOTE:
Models defined in Pytorch expect 2D image data in the dimensions (batch size, channels (colours), height of the image, width of the image)

#Create Dataloader
The 'init' function initialises the dataloader. This class is responsible for loading the datasets. It takes the 'root_dir', 'task', 'plane', 'train' and 'transform' as input parameters.
root_dir - the directory to where the data is stored.

task - whether the model is being trained to detect acl tears, meniscus tears or abnormalities. Possible values are 'acl', 'meniscus' or 'abnormal'.

plane - whether the model is being trained on axial, coronal or sagittal data. Possible values are 'axial', 'coronal' or 'sagittal'.

train - is this the dataloader for the training data or the validation data. Possible values are 'True' to load training data or 'False' to load validation data.

transform - a compose function for performing transformations to the images.

The init function creates 1) a list of paths to each MRI, 2) a corresponding list of labels that are either ones or zeros and 3) weights.


---



The __len__ function returns the length of the dataset.


---
The __getitem__ function is iteratively called throughout the training process. It takes an index as a input parameter. It loads the MRI at the given index from the list of paths defined in the init function. It also returns the label and weight for the MRI at that index.



In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=False, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train=train
        if self.train == True:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform

        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index]) #load MRI
        label = self.labels[index] #get label of MRI
        label = torch.FloatTensor([label]) #convert type from numpy to torch

        if self.transform: #if you are transforming it
            array = self.transform(array) #transform the image
            array = array.numpy()


        array = np.stack((array,)*3, axis=1) #the model expects dimensions of (3, 256, 256), the MRIs are greyscale of size (256, 256). Therefore, we stack the image three times to fit the dimensions for the model.
        array = torch.FloatTensor(array)

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        return array, label, weight

#Train the model
##Define variables
**TO DO:** Change directory to where you store your data. Use the toolbar to the side of this page to view your file system.

In [ ]:
directory='./gdrive/MyDrive/MRNet/data/' #directory to the data
task = 'acl'
plane = 'sagittal'
lr = 1e-5 #learning rate
num_epochs = 50 # number of epochs

##Initialise the model, optimiser, scheduler, transformations and data loader.

In [ ]:

model = Net() #initialise the model
if torch.cuda.is_available(): #if there is a GPU available, put the model on the GPU
    model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr= lr, weight_decay=0.1) #define the optimiser as Adam

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=4, factor=.3, threshold=1e-4, verbose=True) #define a scheduler that decreases the learning rate if there has been no reduction in validation loss is four epochs

#define a compose function that is a series of transformations on the images.
augmentor = Compose([
            transforms.Lambda(lambda x: torch.Tensor(x)), #converts from numpy to tensor
            RandomRotate(25), #rotate the image by 25 degrees
            RandomTranslate([0.11, 0.11]), #blur the edges
            RandomFlip(), #flip the image
        ])

#initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
train_dataset = Dataset(directory, task, plane,
                         train=True, transform=augmentor)
valid_dataset = Dataset(
      directory, task, plane, train=False, transform = None)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=1, shuffle=True, num_workers=2, drop_last=False)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

##Training Loop

In [ ]:
early_trigger = 10 #if the validation AUC hasn't increased in ten epochs, stop the training
early_stop = 0 #counter for the number of iterations where there has been no increase in validation AUC
best_val_auc = 0
#for loop for each epoch
for epoch in range(num_epochs):
      #get learning rate
      current_lr = lr

      y_preds = []
      y_trues = []
      losses = []
      _ = model.train()
      #loop through each MRI in the training set
      for i, (image, label, weight) in enumerate(train_loader):
          optimizer.zero_grad()

          #load all data onto the GPU
          if torch.cuda.is_available():
              image = image.cuda()
              label = label.cuda()
              weight = weight.cuda()

          label = label[0]
          weight = weight[0]

          #pass the MRI through the model
          prediction = model.forward(image.float()).squeeze(0)

          #calculate the loss
          loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
          loss.backward() #back propagation
          optimizer.step()

          loss_value = loss.item()
          losses.append(loss_value)

          probas = torch.sigmoid(prediction) #convert output of model (logits) to a value between zero and one. This can be interpretted as a probability

          y_trues.append(int(label[0]))
          y_preds.append(probas[0].item())

          try:
              auc = metrics.roc_auc_score(y_trues, y_preds)
          except:
              auc = 0.5

          train_loss = np.round(np.mean(losses), 4)
          train_auc = np.round(auc, 4)

      #evaluate the model on the validation data after each epoch
      _ = model.eval()
      y_trues = []
      y_preds = []
      losses = []
      for i, (image, label, weight) in enumerate(valid_loader):

        if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
            weight = weight.cuda()

        label = label[0]
        weight = weight[0]

        prediction = model.forward(image.float()).squeeze(0)

        loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)

        loss_value = loss.item()
        losses.append(loss_value)

        probas = torch.sigmoid(prediction)

        y_trues.append(int(label[0]))
        y_preds.append(probas[0].item())

        try:
            auc = metrics.roc_auc_score(y_trues, y_preds)
        except:
            auc = 0.5

        val_loss = np.round(np.mean(losses), 4)
        val_auc = np.round(auc, 4)

      if val_auc > best_val_auc:
        best_val_auc = val_auc
        early_stop=0
      else:
        early_stop+= 1

      if early_stop == early_trigger:
        print('Early stopping after {} epochs'.format(epoch))
        sys.exit()
      scheduler.step(val_loss)

      print("epoch : {0} | train loss : {1} | train auc {2} | val loss {3} | val auc {4} ".format(
          epoch, train_loss, train_auc, val_loss, val_auc))


      print('-' * 30)

epoch : 0 | train loss : 1.1598 | train auc 0.5539 | val loss 0.8801 | val auc 0.6588 
------------------------------
epoch : 1 | train loss : 1.0492 | train auc 0.6773 | val loss 0.9639 | val auc 0.7135 
------------------------------
epoch : 2 | train loss : 0.9052 | train auc 0.7975 | val loss 0.77 | val auc 0.7635 
------------------------------
epoch : 3 | train loss : 0.7987 | train auc 0.8483 | val loss 0.7509 | val auc 0.8058 
------------------------------
epoch : 4 | train loss : 0.7384 | train auc 0.8734 | val loss 1.3227 | val auc 0.8129 
------------------------------
epoch : 5 | train loss : 0.6776 | train auc 0.8969 | val loss 0.7783 | val auc 0.7988 
------------------------------
epoch : 6 | train loss : 0.606 | train auc 0.9202 | val loss 0.9586 | val auc 0.7983 
------------------------------
epoch : 7 | train loss : 0.5338 | train auc 0.9399 | val loss 0.6189 | val auc 0.8614 
------------------------------
epoch : 8 | train loss : 0.4995 | train auc 0.9463 | val lo